# Harmonization across studies with PIC-SURE

This tutorial notebook will demonstrate how to query and work with the BioData Catalyst studies, particularly cross-study harmonization. For a more step-by-step introduction to the python PIC-SURE API, see the `1_PICSURE_API_101.ipynb` notebook.

**Before running this notebook, please be sure to review the \"Get your security token\" documentation, which exists in the NHLBI_BioData_Catalyst [README.md file](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/tree/master/NHLBI_BioData_Catalyst#get-your-security-token). It explains about how to get a security token, which is mandatory to access the databases.**

-----

# Environment set-up

### Pre-requisites
- R 3.4 or later


### Install packages
Install the following:
- packages listed in the `requirements.R` file
- PIC-SURE API components (from Github)
    - PIC-SURE Adapter
    - PIC-SURE Client

In [ ]:
source("R_lib/requirements.R")

In [ ]:
Sys.setenv(TAR = "/bin/tar")
options(unzip = "internal")
install.packages("https://cran.r-project.org/src/contrib/Archive/devtools/devtools_1.13.6.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/R6_2.5.1.tar.gz", repos=NULL, type="source")
install.packages("https://cran.r-project.org/src/contrib/hash_2.2.6.1.tar.gz", repos=NULL, type="source")
install.packages(c("urltools"),repos = "http://cran.us.r-project.org")
devtools::install_github("hms-dbmi/pic-sure-r-client", force=T)
devtools::install_github("hms-dbmi/pic-sure-r-adapter-hpds", force=T)
devtools::install_github("hms-dbmi/pic-sure-biodatacatalyst-r-adapter-hpds", force=T)

##### Loading user-defined functions

In [ ]:
source("R_lib/utils.R")

## Connecting to a PIC-SURE resource

In [ ]:
PICSURE_network_URL <- "https://picsure.biodatacatalyst.nhlbi.nih.gov/picsure"
resource_id <- "02e23f52-f354-4e8b-992c-d37c8b9ba140"
token_file <- "token.txt"

In [ ]:
token <- scan(token_file, what = "character")

In [ ]:
connection <- picsure::connect(url = PICSURE_network_URL,
                                 token = token)
resource <- bdc::get.resource(connection,
                               resourceUUID = resource_id)

-----

## Harmonizing variables with PIC-SURE
One of the key challenges to conducting analyses with several studies is ensuring correct data harmonization, or combining of data from different sources. There are many harmonization techniques, but this notebook will demonstrate how to find and extract similar variables from different studies in PIC-SURE. Two examples of this will be shown:
1. Retrieving variables for *sex and gender* across studies with BMI
2. Harmonizing the variable *"orthopnea"* across studies with age


*For more information about the TOPMed DCC Harmonized Data Set in PIC-SURE, please refer to the [`2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb`](https://github.com/hms-dbmi/Access-to-Data-using-PIC-SURE-API/blob/master/NHLBI_BioData_Catalyst/R/2_TOPMed_DCC_Harmonized_Variables_analysis.ipynb) notebook*

-----

### Sex and gender variables across studies

Let's start by doing separate searches for `sex` and `gender` to gain a better understanding of the variables that exist in PIC-SURE with these terms.

In [ ]:
full_dict <- bdc::find.in.dictionary(resource) %>%
    bdc::extract.entries()
full_multiindex_dict <- get_multiIndex_variablesDict(full_dict)

In [ ]:
sex_mask <- full_multiindex_dict$simplified_name[str_detect(full_multiindex_dict$simplified_name, 'sex')]
sex <- full_multiindex_dict[full_multiindex_dict$simplified_name %in% sex_mask,]
gender_mask <- full_multiindex_dict$simplified_name[str_detect(full_multiindex_dict$simplified_name, 'gender')]
gender <- full_multiindex_dict[full_multiindex_dict$simplified_name %in% gender_mask,]

In [ ]:
# Uncomment the following lines of code to preview the filtered dataframes
#head(sex)
#head(gender)

After reviewing the variables using the dataframe (or the [user interface](https://picsure.biodatacatalyst.nhlbi.nih.gov/psamaui/login)), let's say we are interested in sex/gender variables from the following studies:
- TOPMed Harmonized data set
- ECLIPSE (Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints)
- EOCOPD (Early Onset of COPD)

However, these concept paths are labelled differently for each of these studies. For example, some use the keyword `sex` while others use `gender`. To acccount for these differences, we need to develop a way to search for multiple keywords at once.

First, let's get all of the concept paths associated with each study.

In [ ]:
topmed_harmonized <- bdc::find.in.dictionary(resource, 'DCC Harmonized data set') %>%
    bdc::extract.entries()
eclipse <- bdc::find.in.dictionary(resource, 'Evaluation of COPD Longitudinally to Identify Predictive Surrogate Endpoints (ECLIPSE)') %>%
    bdc::extract.entries()
eocopd <- bdc::find.in.dictionary(resource, 'NHLBI TOPMed: Boston Early-Onset COPD Study') %>%
    bdc::extract.entries()

In [ ]:
topmed_harmonized

Now we will search for the terms of interest (`sex` and `gender`) and filter out these concept paths.

Below is a simple user-defined function that you could use to accomplish this.

In [ ]:
# Function that filters out variables from a dataframe (df) that contain any of the terms (list_of_terms)
find_vars <- function(df, list_of_terms){
    if(length(list_of_terms)>1){
        pattern <- paste(list_of_terms, collapse='|')
    } else {
        pattern <- list_of_terms
    }
    pattern <- paste(c('(', pattern, ')'), collapse = '')
    df_filtered <- df %>%
        filter(grepl(pattern, name, ignore.case=TRUE))
    vars_list <- list(df_filtered$name)[[1]]
    return(vars_list)
}

In [ ]:
# Search for 'sex' and 'gender' variables in TOPMed Harmonized dataset
topmed_var <- find_vars(topmed_harmonized, c('sex', 'gender'))
print(c("Concept path from TOPMed Harmonized data set:", topmed_var))

In [ ]:
# Search for 'sex' and 'gender' variables in ECLIPSE dataset
eclipse_var <- find_vars(eclipse, c('sex', 'gender'))
print(c("Concept path from ECLIPSE data set:", eclipse_var))

In [ ]:
# Search for 'sex' and 'gender' variables in EOCOPD dataset
eocopd_vars <- find_vars(eocopd, c('sex', 'gender'))
print(c("Number of concept paths from EOCOPD data set:", length(eocopd_vars)))

Since there are multiple concept paths that are contain either `gender` or `sex` in the EOCOPD dataset, we can investigate these concept paths to determine the true variable of interest.

In [ ]:
# Uncomment the following line to see full list of sex/gender variables in EOCOPD
#print(c("Full list of variables", eocopd_vars))

# Based on this, we can see that the variable we want for this analysis is the last in the list: Gender of participant
eocopd_var <- find_vars(eocopd, c('gender of participant'))
print(c("Concept path from EOCOPD data set:", eocopd_var))

As part of our research, let's also say we are interested in body mass index (BMI) measurements across these studies. Let's save these concept paths to use in our queries as well.

In [ ]:
topmed_bmi <- find_vars(topmed_harmonized, c('body mass index'))
print(topmed_bmi)
eclipse_bmi <- find_vars(eclipse, c('body mass index'))
print(eclipse_bmi)
eocopd_bmi <- find_vars(eocopd, c('body mass index'))
print(eocopd_bmi)

Now that we know and have saved our concept paths of interest, we can use these to build our query. 

**Note: queries with the TOPMed DCC Harmonized data set cannot be combined with concept paths from other datasets. Because of this, we will run separate queries and combine the dataframes.**

In [ ]:
# Initialize a query
eclipse_query <- bdc::new.query(resource = resource)
# Build query using these concept paths
bdc::query.anyof.add(query = eclipse_query,
                     keys = eclipse_var)
bdc::query.anyof.add(query = eclipse_query,
                     keys = eclipse_bmi)

In [ ]:
# Check results
eclipse_results <- bdc::query.run(eclipse_query, result.type = 'dataframe')
head(eclipse_results)

In [ ]:
# Initialize a query
eocopd_query <- bdc::new.query(resource = resource)
# Build query using these concept paths
bdc::query.anyof.add(query = eocopd_query,
                     keys = eocopd_var)
bdc::query.anyof.add(query = eocopd_query,
                     keys = eocopd_bmi)

In [ ]:
# Check results
eocopd_results <- bdc::query.run(eocopd_query, result.type = 'dataframe')
head(eocopd_results)

In [ ]:
# Initialize a query
dcc_harmonized_query <- bdc::new.query(resource = resource)
# Build query using these concept paths
bdc::query.anyof.add(query = dcc_harmonized_query,
                     keys =  topmed_var)
bdc::query.anyof.add(query = dcc_harmonized_query,
                     keys = topmed_bmi)

In [ ]:
# Check results
dcc_harmonized_results <- bdc::query.run(dcc_harmonized_query, result.type = 'dataframe')
head(dcc_harmonized_results)

Now that we have our patient-level dataframes, we can combine them into a single, cohesive dataframe.

The following function accomplishes three main tasks:
1. Removes extra columns, such as consent information
2. Renames the BMI and Sex columns
3. Adds the Dataset column, which corresponds to the study

In [ ]:
clean_up_df <- function(df, study){
    columns_to_drop <- c('\\_Parent Study Accession with Subject ID\\', '\\_Topmed Study Accession with Subject ID\\', '\\_consents\\', '\\_harmonized_consent\\')
    df1 <- df[,!(names(df) %in% columns_to_drop)]
    if(grepl('body mass index', names(df1)[2], ignore.case=TRUE)){
        names(df1)[2] <- 'BMI'
        names(df1)[3] <- 'Sex'
    } else {
        names(df1)[2] <- "Sex"
        names(df1)[3] <- "BMI"
    }
    df2 <- na.omit(df1)
    df2$Dataset <- study
    return(df2)
}

In [ ]:
clean_eclipse <- clean_up_df(eclipse_results, 'ECLIPSE')
clean_eocopd <- clean_up_df(eocopd_results, "EOCOPD")
clean_dcc <- clean_up_df(dcc_harmonized_results, "TOPMed Harmonized")

The datasets have been prepped. We can now merge them and begin our analysis.

In [ ]:
# Combine individual dataframes
final_df <- rbind(clean_eclipse, clean_eocopd, clean_dcc)

In [ ]:
# Comparison of the datasets and sample harmonization
separate <- final_df %>%
    select(-c('Patient ID')) %>%
    group_by(Dataset, Sex) %>%
    summarise(mean_BMI = mean(BMI))
print(separate)

harmonized <- final_df %>%
    select(-c('Patient ID', 'Dataset')) %>%
    group_by(Sex) %>%
    summarise(mean_BMI = mean(BMI))
print(harmonized)

In [ ]:
# Make lists of male and female mean BMI for plotting
male_means <- c(separate$mean_BMI[separate$Sex == 'Male'], harmonized$mean_BMI[harmonized$Sex == 'Male'])
female_means <-  c(separate$mean_BMI[separate$Sex == 'Female'], harmonized$mean_BMI[harmonized$Sex == 'Female'])

In [ ]:
studies <- rep(c('ECLIPSE', "EOCOPD", 'TOPMed Harmonized', "Combined"), 2)
sex <- c(rep('Male', 4), rep('Female', 4))
total_means <- c(male_means, female_means)
to_plot <- data.frame(total_means, sex, studies)
to_plot

In [ ]:
ggplot(data = to_plot, aes(fill=studies, y=total_means, x=sex)) + 
    geom_bar(position="dodge", stat="identity") +
    labs(title = "Average Body Mass Index by Sex and Dataset",
         x = "Sex", y = "Body Mass Index (BMI)",
         fill = "Dataset")